<a href="https://colab.research.google.com/github/08-pa/TWITTER-SENTIMENT-ANALYSIS/blob/main/TWITTER_SENTIMENT_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 93% 75.0M/80.9M [00:00<00:00, 89.0MB/s]
100% 80.9M/80.9M [00:00<00:00, 98.9MB/s]


In [ ]:
from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r')as zip:
  zip.extractall()
  print('the dataset is extracted')

the dataset is extracted


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#printing stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
coloum_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=coloum_names,  encoding='ISO-8859-1')

In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head(5)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [ ]:
#0-->nagative tweet
#1-->positive tweet
#steming is process to reducing word to rootword
port_stem=PorterStemmer()

In [ ]:
twitter_data['stemmed_coport'] = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = content.lower()
    stemmed_content = stemmed_content.split()
    stemmer = nltk.SnowballStemmer('english')
    stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in nltk.corpus.stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content




In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_coport,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",<PorterStemmer>,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,<PorterStemmer>,upset can't updat facebook text it... might cr...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,<PorterStemmer>,@kenichan dive mani time ball. manag save 50% ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,<PorterStemmer>,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",<PorterStemmer>,"@nationwideclass no, behav all. i'm mad. here?..."


In [ ]:
print(twitter_data['stemmed_content'])

0          @switchfoot http://twitpic.com/2y1zl - awww, t...
1          upset can't updat facebook text it... might cr...
2          @kenichan dive mani time ball. manag save 50% ...
3                            whole bodi feel itchi like fire
4          @nationwideclass no, behav all. i'm mad. here?...
                                 ...                        
1599995                       woke up. school best feel ever
1599996    thewdb.com - cool hear old walt interviews! â...
1599997                      readi mojo makeover? ask detail
1599998    happi 38th birthday boo alll time!!! tupac ama...
1599999    happi #charitytuesday @thenspcc @sparkschar @s...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
print(X)

['@switchfoot http://twitpic.com/2y1zl - awww, that bummer. shoulda got david carr third day it. ;d'
 "upset can't updat facebook text it... might cri result school today also. blah!"
 '@kenichan dive mani time ball. manag save 50% rest go bound' ...
 'readi mojo makeover? ask detail'
 'happi 38th birthday boo alll time!!! tupac amaru shakur'
 'happi #charitytuesday @thenspcc @sparkschar @speakinguph4h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)


['watch saw iv drink lil wine' "@hatermagazin i'm in!"
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 "hope everyon mother great day! can't wait hear guy store tomorrow!"
 'love wake folgers. bad voic deeper his.']


In [ ]:
print(X_test)

['@mmangen - fine! much time chat twitter. hubbi back summer &amp; tend domin free time.'
 'ah may show w/ ruth kim &amp; geoffrey sanhueza'
 '@ishatara mayb bay area thang dammit!' ...
 '@destini41 nevertheless hooray! 4700 member wonder safe trip!'
 'feel well' '@supersandro !!!! thank you!']


In [ ]:
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.fit_transform(X_test)


In [ ]:
print(X_train)

  (0, 555056)	0.44872071677182435
  (0, 311521)	0.42139878820156595
  (0, 166661)	0.3784258390024876
  (0, 253881)	0.5248770996321014
  (0, 452408)	0.3579598085710012
  (0, 547836)	0.2724618684273102
  (1, 246930)	0.4216748669629444
  (1, 225963)	0.9067471017719236
  (2, 374825)	0.1668935707272662
  (2, 193530)	0.20185026966419675
  (2, 214505)	0.18696806053652665
  (2, 245500)	0.1610871849298624
  (2, 515944)	0.15260651817586954
  (2, 348922)	0.24083670228348006
  (2, 555463)	0.33615513831638094
  (2, 129186)	0.31180310787000864
  (2, 544237)	0.32797662458056626
  (2, 512826)	0.3201678621219188
  (2, 190292)	0.291960339536279
  (2, 513806)	0.1860264155402729
  (2, 184225)	0.19027374838761577
  (2, 166661)	0.46136477277006543
  (3, 239237)	0.38347641261447546
  (3, 518639)	0.2688968511993747
  (3, 492216)	0.39148958119338084
  :	:
  (1279996, 409476)	0.21030232633902124
  (1279996, 314914)	0.22194518109414565
  (1279996, 377841)	0.17758989770758854
  (1279996, 519819)	0.186817733790113

In [ ]:
print(X_test)

  (0, 74464)	0.23600042605384178
  (0, 59832)	0.37162309628249196
  (0, 190454)	0.3467732063658612
  (0, 17979)	0.17154907194831126
  (0, 185480)	0.22002452821015506
  (0, 25203)	0.1620660519412641
  (0, 89228)	0.2879507167548526
  (0, 200357)	0.18072887309386862
  (0, 41969)	0.2689822191252364
  (0, 194827)	0.3181046189235982
  (0, 135795)	0.1783057377161917
  (0, 71769)	0.25568678593483674
  (0, 132135)	0.4422148143454971
  (1, 168936)	0.50795001914606
  (1, 77835)	0.4808310203180851
  (1, 108500)	0.35298540164971204
  (1, 167067)	0.42422111797277623
  (1, 175240)	0.2114392750889712
  (1, 125787)	0.24411549387967696
  (1, 14374)	0.2634232057602148
  (1, 17979)	0.1803755263394934
  (2, 52464)	0.37210228108808885
  (2, 191250)	0.4448975214167908
  (2, 21242)	0.35917845494725115
  (2, 26842)	0.388459095068717
  :	:
  (319995, 60681)	0.30037197234040824
  (319995, 211041)	0.2861830346457791
  (319995, 61394)	0.2682755640405168
  (319995, 156367)	0.33006189231157157
  (319995, 82276)	0.24

In [ ]:
#logistic regression
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(y_train, X_train_prediction)

In [ ]:
print("accurancy score on the training data",training_data_accuracy)

accurancy score on the training data 0.816740625


In [ ]:
if X_test.shape[1] == model.n_features_in_:
    X_test_prediction = model.predict(X_test)
    training_data_accuracy = accuracy_score(y_test, X_test_prediction)

In [ ]:
print("accurancy score on the training data",training_data_accuracy)

accurancy score on the training data 0.816740625


model accurancy=81.6%

In [ ]:
import pickle

In [ ]:
filename='trained_model.sav'
pickle.dump(model,open(filename, 'wb'))

In [ ]:
#using save model for future prediction
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
print(X_new.shape)

(1, 218448)


In [ ]:
print(model.n_features_in_)

577933


In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:

prediction = [1]


if (prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

Positive Tweet


In [ ]:
prediction = [300]


if (prediction[0] == 0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

Positive Tweet
